# RNN - LSTM

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from project_1.config import PROJ_ROOT, PROCESSED_DATA_DIR

## Load _final.parquet files

In [6]:
# Load the data from Parquet files
sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Define file names
file_names = ["Outcomes-a.txt", "Outcomes-b.txt", "Outcomes-c.txt"]

# Directory path
base_path = PROJ_ROOT / "data" / "data_1" / "predicting-mortality-of-icu-patients-the-physionet-computing-in-cardiology-challenge-2012-1.0.0"

# Read files into DataFrames containing all variables
outcomes_a, outcomes_b, outcomes_c = [pd.read_csv(base_path / name) for name in file_names]

# Extract only the "RecordID" and "In-hospital_death" column into separate DataFrames
death_a, death_b, death_c = [df[["RecordID", "In-hospital_death"]] for df in [outcomes_a, outcomes_b, outcomes_c]]
print(death_a.shape)

#CHECK for missing values in the outcome data
"""print(death_a.isnull().sum())
print(death_b.isnull().sum())
print(death_c.isnull().sum())"""
# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head()

(4000, 2)
(183416, 43)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Urine,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.950526,-0.23008,-0.596332,1.671639,-0.013487,-0.832594,-0.109176,...,11.571429,0.753623,1.125,0.0,1.923077,0.132075,-0.176471,0.450704,1.545455,0.285714
1,132539.0,2025-03-10 01:00:00,0.0,-0.950526,-0.23008,-0.596332,1.967793,0.172112,-0.608431,-0.109176,...,2.857143,-0.420290,0.125,0.0,-0.246154,0.000000,-0.294118,0.436620,0.000000,-0.126984
2,132539.0,2025-03-10 02:00:00,0.0,-0.950526,-0.23008,-0.596332,-1.734132,0.125712,0.848629,0.830987,...,-0.357143,-0.014493,-0.875,0.0,0.000000,0.773585,-0.205882,-0.380282,0.181818,-0.095238
3,132539.0,2025-03-10 03:00:00,0.0,-0.950526,-0.23008,-0.596332,1.523562,0.380910,-0.832594,-0.579257,...,0.642857,0.188406,-0.375,0.0,0.215385,-0.698113,-0.588235,1.126761,-0.181818,4.650794
4,132539.0,2025-03-10 04:00:00,0.0,-0.950526,-0.23008,-0.596332,0.487023,-0.964680,1.483758,-0.814297,...,-0.142857,-1.144928,1.000,0.0,2.738462,-0.490566,-0.558824,-0.225352,0.363636,0.904762


In [92]:
# Check if ICUType is present
print(sets_dict["set_a"].ICUType.unique())

[ 0.5 -0.5  0.  -1. ]


In [108]:
# Remove ICUType from the data
if "ICUType" in sets_dict["set_a"].columns:
    sets_dict["set_a"] = sets_dict["set_a"].drop(columns=["ICUType"])

if "ICUType" in sets_dict["set_b"].columns:
    sets_dict["set_b"] = sets_dict["set_b"].drop(columns=["ICUType"])

if "ICUType" in sets_dict["set_c"].columns:
    sets_dict["set_c"] = sets_dict["set_c"].drop(columns=["ICUType"])

## Basic Implementation of LSTM

In [94]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]             # Take last time step: (batch_size, hidden_size)
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

# Pre process Dataframe => Tensor

In [109]:
# Only convert sets A and C

# Convert the sets to PyTorch tensors
features_cols = [col for col in sets_dict["set_a"].columns if col not in ["RecordID", "Time"]]

sequences_a = []
sequences_c = []
set_a = sets_dict["set_a"]
set_c = sets_dict["set_c"]
for record_id, group in set_a.groupby("RecordID"):
    seq = group[features_cols].to_numpy(dtype=np.float32)
    sequences_a.append(torch.tensor(seq))

for record_id, group in set_c.groupby("RecordID"):
    seq = group[features_cols].to_numpy(dtype=np.float32)
    sequences_c.append(torch.tensor(seq))

# Now sequences_a is a list of PyTorch tensors, of shape (48, 41) each (48 timesteps, 41 features)
sequences_a[0].shape, sequences_c[0].shape

(torch.Size([48, 40]), torch.Size([47, 40]))

In [96]:
# Check if all sequences have the same length
lengths = [len(seq) for seq in sequences_a]
print(len(lengths))

4000


It makes sense, we do not have complete 49 measurements for each patient, they might be less

### Need to pad the sequences

In [110]:
from torch.nn.utils.rnn import pad_sequence

padded_sequences_a = pad_sequence(sequences_a, batch_first=True)
padded_sequences_c = pad_sequence(sequences_c, batch_first=True)

train_X = padded_sequences_a
train_y = torch.tensor(death_a["In-hospital_death"])

test_X = padded_sequences_c
test_y = torch.tensor(death_c["In-hospital_death"])

train_X.shape, train_y.shape, test_X.shape, test_y.shape

(torch.Size([4000, 49, 40]),
 torch.Size([4000]),
 torch.Size([4000, 49, 40]),
 torch.Size([4000]))

# Create DataLoaders (very useful for NN)

In [111]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [112]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTM_Model(
    input_size=train_X.shape[2],  # 40
    hidden_size=64,
    num_layers=1,
    num_classes=1,
    dropout=0.0
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)

        # Sanity check shapes
        assert outputs.shape == batch_y.shape, f"Shape mismatch: {outputs.shape} vs {batch_y.shape}"

        # Convert to float
        batch_y = batch_y.float()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/10 | Loss: 0.4836
Epoch 2/10 | Loss: 0.3410
Epoch 3/10 | Loss: 0.3102
Epoch 4/10 | Loss: 0.2930
Epoch 5/10 | Loss: 0.2774
Epoch 6/10 | Loss: 0.2672
Epoch 7/10 | Loss: 0.2530
Epoch 8/10 | Loss: 0.2472
Epoch 9/10 | Loss: 0.2378
Epoch 10/10 | Loss: 0.2141


# Test Time

In [113]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import torch.nn.functional as F

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Forward pass
        logits = model(batch_X)
        probs = torch.sigmoid(logits)  # convert to [0, 1]

        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Thresholding for accuracy (default: 0.5)
binary_preds = (all_preds >= 0.5).astype(int)

# Metrics
roc_auc = roc_auc_score(all_labels, all_preds)
auprc = average_precision_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, binary_preds)

print(f"\n📊 Test Metrics:")
print(f"✅ ROC AUC:  {roc_auc:.3f}")
print(f"✅ AUPRC:    {auprc:.3f}")
print(f"✅ Accuracy: {accuracy:.3f}")


📊 Test Metrics:
✅ ROC AUC:  0.814
✅ AUPRC:    0.450
✅ Accuracy: 0.860


## Attempt 1 - Mean Pooling in LSTM

In [114]:
class LSTM_Model_Pooling(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Pooling, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out.mean(dim=1)           # Pooling: (batch_size, hidden_size)   
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

## Train Loop

In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTM_Model_Pooling(
    input_size=train_X.shape[2],  # 40
    hidden_size=64,
    num_layers=1,
    num_classes=1,
    dropout=0.0
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)

        # Sanity check shapes
        assert outputs.shape == batch_y.shape, f"Shape mismatch: {outputs.shape} vs {batch_y.shape}"

        # Convert to float
        batch_y = batch_y.float()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/10 | Loss: 0.4719
Epoch 2/10 | Loss: 0.3476
Epoch 3/10 | Loss: 0.3278
Epoch 4/10 | Loss: 0.3129
Epoch 5/10 | Loss: 0.3036
Epoch 6/10 | Loss: 0.3003
Epoch 7/10 | Loss: 0.2832
Epoch 8/10 | Loss: 0.2678
Epoch 9/10 | Loss: 0.2595
Epoch 10/10 | Loss: 0.2517


In [116]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import torch.nn.functional as F

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Forward pass
        logits = model(batch_X)
        probs = torch.sigmoid(logits)  # convert to [0, 1]

        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Thresholding for accuracy (default: 0.5)
binary_preds = (all_preds >= 0.5).astype(int)

# Metrics
roc_auc = roc_auc_score(all_labels, all_preds)
auprc = average_precision_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, binary_preds)

print(f"\n📊 Test Metrics:")
print(f"✅ ROC AUC:  {roc_auc:.3f}")
print(f"✅ AUPRC:    {auprc:.3f}")
print(f"✅ Accuracy: {accuracy:.3f}")


📊 Test Metrics:
✅ ROC AUC:  0.809
✅ AUPRC:    0.443
✅ Accuracy: 0.855


# Attempt 2 - Max Pooling

# Bidirection LSTM

In [117]:
class LSTM_Model_Bi(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=1, dropout=0.3):
        super(LSTM_Model_Bi, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,       # 41 features per time step
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes) # *2 for bidirectional

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)           # out: (batch_size, seq_len, hidden_size)
        out = out.max(dim=1)[0]
        out = self.fc(out)              # (batch_size, num_classes)
        return out.squeeze()            # (batch_size,) for BCEWithLogitsLoss

In [118]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTM_Model_Bi(
    input_size=train_X.shape[2],  # 41
    hidden_size=16,
    num_layers=1,
    num_classes=1,
    dropout=0.0
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)

        # Sanity check shapes
        assert outputs.shape == batch_y.shape, f"Shape mismatch: {outputs.shape} vs {batch_y.shape}"

        # Convert to float
        batch_y = batch_y.float()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/10 | Loss: 0.5216
Epoch 2/10 | Loss: 0.3950
Epoch 3/10 | Loss: 0.3776
Epoch 4/10 | Loss: 0.3619
Epoch 5/10 | Loss: 0.3439
Epoch 6/10 | Loss: 0.3311
Epoch 7/10 | Loss: 0.3159
Epoch 8/10 | Loss: 0.3054
Epoch 9/10 | Loss: 0.2938
Epoch 10/10 | Loss: 0.2840


In [119]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import torch.nn.functional as F

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Forward pass
        logits = model(batch_X)
        probs = torch.sigmoid(logits)  # convert to [0, 1]

        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Thresholding for accuracy (default: 0.5)
binary_preds = (all_preds >= 0.5).astype(int)

# Metrics
roc_auc = roc_auc_score(all_labels, all_preds)
auprc = average_precision_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, binary_preds)

print(f"\n📊 Test Metrics:")
print(f"✅ ROC AUC:  {roc_auc:.3f}")
print(f"✅ AUPRC:    {auprc:.3f}")
print(f"✅ Accuracy: {accuracy:.3f}")


📊 Test Metrics:
✅ ROC AUC:  0.805
✅ AUPRC:    0.432
✅ Accuracy: 0.858


# Trying to use MinMaxScaler

In [143]:
# Import data before being scaled
sets_dict_2 = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_to_scale.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict_2[f"set_{set_name}"] = temp_set

print(sets_dict_2["set_a"].shape)
sets_dict_2["set_a"].head()

(183416, 43)


,Time,RecordID,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,2025-03-10 00:00:00,132539.0,54.0,15.0,2.9,15.0,0.0,89.0,23.0,36.40,...,36.0,258.0,7.48,47.0,87.00,98.0,89.0,3.60,152.0,3.5
1,2025-03-10 01:00:00,132539.0,54.0,17.0,1.0,15.0,0.0,147.0,23.0,29.60,...,43.0,385.0,7.40,50.0,96.50,98.0,135.0,3.25,160.0,0.9
2,2025-03-10 02:00:00,132539.0,54.0,20.0,0.8,15.0,0.0,210.0,27.0,29.80,...,35.0,195.0,7.32,69.5,55.00,97.0,130.0,6.20,158.0,1.1
3,2025-03-10 03:00:00,132539.0,54.0,8.0,0.7,15.0,0.0,75.0,21.0,26.75,...,62.0,285.0,7.36,47.0,85.00,98.0,105.0,6.40,169.0,31.0
4,2025-03-10 04:00:00,132539.0,54.0,9.0,0.3,15.0,0.0,154.0,20.0,33.70,...,45.0,431.0,7.47,78.0,85.75,99.0,136.0,2.30,111.0,7.4


In [144]:
# Cbeck Gender
print(sets_dict_2["set_a"]["Gender"].sum())

103094.6


In [145]:
# Now use the MinMaxScaler on the selected columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
selected_columns = [col for col in sets_dict_2["set_a"].columns if col not in ["RecordID", "Time"]]
sets_dict_2["set_a"][selected_columns] = scaler.fit_transform(sets_dict_2["set_a"][selected_columns])
sets_dict_2["set_b"][selected_columns] = scaler.transform(sets_dict_2["set_b"][selected_columns])
sets_dict_2["set_c"][selected_columns] = scaler.transform(sets_dict_2["set_c"][selected_columns])

# Check if the scaling was successful
sets_dict_2["set_a"].head()

,Time,RecordID,Age,BUN,Creatinine,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,pH,DiasABP,MAP,SaO2,SysABP,Lactate,Cholesterol,TroponinI
0,2025-03-10 00:00:00,132539.0,0.52,0.076142,0.127273,1.0,0.0,0.069726,0.400000,0.518939,...,0.280899,0.495833,0.034839,0.175373,0.294915,0.972973,0.323636,0.113793,0.410596,0.065440
1,2025-03-10 01:00:00,132539.0,0.52,0.086294,0.040909,1.0,0.0,0.120918,0.400000,0.390152,...,0.359551,0.760417,0.034409,0.186567,0.327119,0.972973,0.490909,0.101724,0.437086,0.012270
2,2025-03-10 02:00:00,132539.0,0.52,0.101523,0.031818,1.0,0.0,0.176523,0.488889,0.393939,...,0.269663,0.364583,0.033978,0.259328,0.186441,0.959459,0.472727,0.203448,0.430464,0.016360
3,2025-03-10 03:00:00,132539.0,0.52,0.040609,0.027273,1.0,0.0,0.057370,0.355556,0.336174,...,0.573034,0.552083,0.034194,0.175373,0.288136,0.972973,0.381818,0.210345,0.466887,0.627812
4,2025-03-10 04:00:00,132539.0,0.52,0.045685,0.009091,1.0,0.0,0.127096,0.333333,0.467803,...,0.382022,0.856250,0.034785,0.291045,0.290678,0.986486,0.494545,0.068966,0.274834,0.145194


In [146]:
# Check Gender column values
print(sets_dict_2["set_a"]["Gender"].unique())

[0.  1.  0.5 0.7 0.4]


Kinda weird isn't it?

In [147]:
# Check if ICUType is present
print(True if "ICUType" in sets_dict_2["set_a"].columns else False)
if "ICUType" in sets_dict_2["set_a"].columns:
    sets_dict_2["set_a"] = sets_dict_2["set_a"].drop(columns=["ICUType"])

if "ICUType" in sets_dict_2["set_b"].columns:
    sets_dict_2["set_b"] = sets_dict_2["set_b"].drop(columns=["ICUType"])

if "ICUType" in sets_dict_2["set_c"].columns:
    sets_dict_2["set_c"] = sets_dict_2["set_c"].drop(columns=["ICUType"])

True


In [148]:
# Only convert sets A and C

# Convert the sets to PyTorch tensors
features_cols = [col for col in sets_dict_2["set_a"].columns if col not in ["RecordID", "Time"]]

sequences_a_2 = []
sequences_c_2 = []
set_a = sets_dict_2["set_a"]
set_c = sets_dict_2["set_c"]
for record_id, group in set_a.groupby("RecordID"):
    seq = group[features_cols].to_numpy(dtype=np.float32)
    sequences_a_2.append(torch.tensor(seq))

for record_id, group in set_c.groupby("RecordID"):
    seq = group[features_cols].to_numpy(dtype=np.float32)
    sequences_c_2.append(torch.tensor(seq))

# Now sequences_a_2 is a list of PyTorch tensors, of shape (48, 41) each (48 timesteps, 41 features)
sequences_a_2[0].shape, sequences_c_2[0].shape

(torch.Size([48, 40]), torch.Size([47, 40]))

In [149]:
from torch.nn.utils.rnn import pad_sequence

padded_sequences_a = pad_sequence(sequences_a_2, batch_first=True)
padded_sequences_c = pad_sequence(sequences_c_2, batch_first=True)

train_X = padded_sequences_a
train_y = torch.tensor(death_a["In-hospital_death"])

test_X = padded_sequences_c
test_y = torch.tensor(death_c["In-hospital_death"])

train_X.shape, train_y.shape, test_X.shape, test_y.shape

(torch.Size([4000, 49, 40]),
 torch.Size([4000]),
 torch.Size([4000, 49, 40]),
 torch.Size([4000]))

In [150]:
# Feed this dataframe into the LSTM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTM_Model(
    input_size=train_X.shape[2],  # 40
    hidden_size=64,
    num_layers=1,
    num_classes=1,
    dropout=0.0
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)

        # Sanity check shapes
        assert outputs.shape == batch_y.shape, f"Shape mismatch: {outputs.shape} vs {batch_y.shape}"

        # Convert to float
        batch_y = batch_y.float()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/10 | Loss: 0.4475
Epoch 2/10 | Loss: 0.3374
Epoch 3/10 | Loss: 0.3134
Epoch 4/10 | Loss: 0.2995
Epoch 5/10 | Loss: 0.2857
Epoch 6/10 | Loss: 0.2710
Epoch 7/10 | Loss: 0.2620
Epoch 8/10 | Loss: 0.2400
Epoch 9/10 | Loss: 0.2282
Epoch 10/10 | Loss: 0.2116


In [151]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
import torch.nn.functional as F

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Forward pass
        logits = model(batch_X)
        probs = torch.sigmoid(logits)  # convert to [0, 1]

        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Thresholding for accuracy (default: 0.5)
binary_preds = (all_preds >= 0.5).astype(int)

# Metrics
roc_auc = roc_auc_score(all_labels, all_preds)
auprc = average_precision_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, binary_preds)

print(f"\n📊 Test Metrics:")
print(f"✅ ROC AUC:  {roc_auc:.3f}")
print(f"✅ AUPRC:    {auprc:.3f}")
print(f"✅ Accuracy: {accuracy:.3f}")


📊 Test Metrics:
✅ ROC AUC:  0.778
✅ AUPRC:    0.406
✅ Accuracy: 0.857
